# Data Reading

In [ ]:
import os
import sys
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Local modules
import nexrad

# iRadar
sys.path.append(os.path.expanduser('~/Developer/iradar/python'))
import iradar

In [ ]:
plt.rcParams['axes.facecolor'] = 'black'

In [ ]:
files = glob.glob(os.path.expanduser('~/Downloads/88d/KDDC/*V06'))

In [ ]:
# filename = os.path.expanduser('~/Downloads/88d/KDDC/KDDC20200131_010529_V06')
# filename = os.path.expanduser('~/Downloads/88d/KDDC/KDDC20200131_165722_V06')
filename = files[0]

azimuth_count = 360
range_count = 800

In [ ]:
sweeps = nexrad.read(filename)

m = sweeps['m'][0, :, :range_count]
z = sweeps['z'][0, :, :range_count]
v = sweeps['v'][0, :, :range_count]

m = np.logical_or(m, z < -5.0)
z[m] = np.nan
v[m] = np.nan

In [ ]:
# The overlay
overlay = iradar.atlas.Overlay((sweeps['longitude'], sweeps['latitude']))
overlay.load()
overlay.setRingRadii([1.0, 30.0, 60.0, 90.0])
overlay.prepareForRect([-50.0, -50.0, 100.0, 100.0])

# The chart
view = iradar.chart.Image(None, None, None, maxrange=80.0, figsize=(600, 675), y=+5, overlay=overlay)

# Data Viewing

In [ ]:
az = sweeps['aa'][0, :]
rr = 1.0e-3 * sweeps['rr'][:range_count]
view.set_data(np.zeros((azimuth_count, range_count)), a=az, r=rr, style='Z')

In [ ]:
im = []

view.set_data(z, style='Z')
im.append(view.pixel_buffer())

view.set_data(v, style='V')
im.append(view.pixel_buffer())

buff = np.array(np.hstack(im), dtype=np.uint8)
mosaic = iradar.chart.PIL.Image.fromarray(buff)

display(mosaic)

In [ ]:
# mosaic.save(os.path.expanduser('~/Downloads/88d/cache/{}.png'.format(os.path.basename(filename))))

In [ ]:
%matplotlib inline

In [ ]:
roll_amount = int(az[0] * 180.0 / np.pi)

In [ ]:
plt.plot(np.roll(az, roll_amount) * 180.0 / np.pi)

In [ ]:
z_rolled = np.roll(z, roll_amount, axis=0)

In [ ]:
zmap = iradar.colormap.zmap()
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', zmap[:, :3], N=len(zmap))

In [ ]:
plt.figure(dpi=250)
plt.imshow(z_rolled[:, 0:300], cmap=cmap)
plt.clim((-32, 96))

In [ ]:
az_sub = np.arange(280, 310)
rr_sub = np.arange(80, 140)

# plt.figure(dpi=200)
# plt.pcolormesh(rr_sub, az_sub, z_rolled[az_sub, :][:, rr_sub], cmap=cmap)
# plt.clim((-32, 96))
# plt.grid()

# Cell Marking

In [ ]:
cells = np.array([
    [288, 110],
    [288, 111],
    [288, 112],
    [288, 113],
    [288, 114],
    [288, 115],
    [288, 116],
    [288, 117],
    [288, 118],
    [288, 119],
    [289, 106],
    [289, 108],
    [289, 109],
    [289, 110],
    [289, 111],
    [289, 112],
    [289, 113],
    [289, 115],
    [289, 116],
    [290, 106],
    [290, 107],
    [290, 108],
    [290, 109],
    [290, 115],
    [290, 116],
    [290, 117],
    [290, 118],
    [290, 119],
    [290, 120],
    [290, 121],
    [290, 122],
    [290, 123],
    [291, 98],
    [291, 99],
    [291, 100],
    [291, 101],
    [291, 102],
    [291, 103],
    [291, 106],
    [291, 107],
    [291, 108],
    [291, 113],
    [291, 114],
    [291, 115],
    [291, 116],
    [291, 117],
    [292, 97],
    [292, 98],
    [292, 99],
    [292, 100],
    [292, 101],
    [292, 102],
    [292, 108],
    [292, 109],
    [292, 110],
    [292, 111],
    [292, 112],
    [292, 113],
    [292, 114],
    [292, 115],
    [292, 116],
    [293, 102],
    [293, 105],
    [293, 106],
    [293, 107],
    [293, 108],
    [293, 109],
    [293, 110],
    [293, 111],
    [293, 112],
    [293, 113],
    [293, 124],
    [293, 125],
    [293, 126],
    [293, 127],
    [293, 128],
    [293, 129],
    [294, 101],
    [294, 102],
    [294, 103],
    [294, 104],
    [294, 105],
    [294, 106],
    [294, 107],
    [294, 108],
    [294, 119],
    [294, 120],
    [294, 121],
    [294, 122],
    [294, 123],
    [294, 124],
    [294, 125],
    [294, 126],
    [295, 97],
    [295, 98],
    [295, 99],
    [295, 100],
    [295, 101],
    [295, 102],
    [295, 103],
    [295, 104],
    [295, 107],
    [295, 112],
    [295, 113],
    [295, 114],
    [295, 115],
    [295, 116],
    [295, 117],
    [295, 118],
    [295, 119],
    [295, 120],
    [296, 94],
    [296, 95],
    [296, 96],
    [296, 97],
    [296, 98],
    [296, 99],
    [296, 100],
    [296, 108],
    [296, 109],
    [296, 110],
    [296, 111],
    [296, 112],
    [296, 113],
    [296, 114],
    [296, 115],
    [297, 104],
    [297, 105],
    [297, 106],
    [297, 107],
    [297, 108],
    [297, 109],
    [297, 110],
    [297, 111],
    [297, 112],
    [297, 118],
    [297, 119],
    [297, 120],
    [297, 121],
    [297, 122],
    [297, 124],
    [298, 100],
    [298, 101],
    [298, 102],
    [298, 103],
    [298, 104],
    [298, 105],
    [298, 106],
    [298, 107],
    [298, 114],
    [298, 115],
    [298, 116],
    [298, 117],
    [298, 118],
    [298, 119],
    [298, 120],
    [298, 121],
    [299, 98],
    [299, 99],
    [299, 100],
    [299, 101],
    [299, 102],
    [299, 103],
    [299, 104],
    [299, 109],
    [299, 110],
    [299, 111],
    [299, 112],
    [299, 113],
    [299, 114],
    [299, 115],
    [300, 105],
    [300, 106],
    [300, 107],
    [300, 108],
    [300, 109],
    [300, 110],
    [300, 111],
    [300, 112],
    [301, 102],
    [301, 104],
    [301, 105],
    [301, 106],
    [301, 107],
    [301, 108],
    [301, 111],
    [302, 100],
    [302, 101],
    [302, 102],
    [302, 103],
    [302, 106],
    [302, 107],
    [303, 101],
    [303, 102],
    [303, 103]
])
z_marked = z_rolled.copy()
for cell in cells:
    z_marked[cell[0], cell[1]] = 65

In [ ]:
plt.figure(dpi=200)
# plt.pcolormesh(rr_sub, az_sub, z_marked[az_sub, :][:, rr_sub], cmap=cmap)
plt.pcolormesh(rr_sub, az_sub, z_rolled[az_sub, :][:, rr_sub], cmap=cmap)
plt.plot(cells[:, 1] + 0.5, cells[:, 0] + 0.5, 'xw')
plt.clim((-32, 96))
plt.grid()

In [ ]:
cells2 = np.array([
    [234, 113],
    [235, 111],
    [235, 112],
    [235, 113],
    [235, 114],
    [235, 115],
    [235, 116],
    [235, 117],
    [236, 112],
    [236, 113],
    [236, 114],
    [236, 115],
    [236, 116],
    [236, 117],
    [236, 118],
    [236, 119],
    [236, 120],
    [236, 121],
    [236, 122],
    [236, 123],
    [237, 115],
    [237, 121],
    [237, 122],
    [237, 123],
    [237, 124],
    [237, 125],
    [238, 114],
    [238, 115],
    [238, 116],
    [238, 117],
    [238, 118],
    [238, 122],
    [238, 124],
    [238, 125],
    [238, 126],
    [238, 127],
    [238, 128],
    [238, 129],
    [239, 108],
    [239, 109],
    [239, 110],
    [239, 111],
    [239, 115],
    [239, 117],
    [239, 118],
    [239, 119],
    [239, 120],
    [239, 121],
    [239, 122],
    [239, 123],
    [239, 124],
    [239, 125],
    [239, 130],
    [239, 131],
    [239, 132],
    [239, 133],
    [239, 135],
    [240, 109],
    [240, 110],
    [240, 112],
    [240, 115],
    [240, 116],
    [240, 117],
    [240, 118],
    [240, 119],
    [240, 120],
    [240, 121],
    [240, 122],
    [240, 123],
    [240, 124],
    [240, 125],
    [240, 126],
    [240, 127],
    [240, 128],
    [240, 134],
    [240, 135],
    [240, 137],
    [241, 110],
    [241, 111],
    [241, 112],
    [241, 113],
    [241, 114],
    [241, 116],
    [241, 119],
    [241, 124],
    [241, 125],
    [241, 126],
    [241, 127],
    [241, 128],
    [241, 135],
    [241, 136],
    [241, 137],
    [241, 138],
    [241, 139],
    [241, 142],
    [241, 135],
    [241, 158],
    [241, 159],
    [241, 160],
    [241, 161],
    [241, 162],
    [241, 163],
    [241, 164],
    [242, 113],
    [242, 114],
    [242, 115],
    [242, 116],
    [242, 117],
    [242, 118],
    [242, 119],
    [242, 120],
    [242, 125],
    [242, 126],
    [242, 127],
    [242, 128],
    [242, 129],
    [242, 130],
    [242, 134],
    [242, 141],
    [242, 142],
    [242, 143],
    [242, 144],
    [242, 145],
    [242, 146],
    [242, 147],
    [242, 148],
    [242, 149],
    [242, 150],
    [242, 151],
    [242, 152],
    [242, 153],
    [242, 154],
    [242, 155],
    [242, 156],
    [242, 157],
    [242, 158],
    [242, 159],
    [242, 160],
    [242, 161],
    [242, 162],
    [242, 163],
    [242, 164],
    [242, 165],
    [242, 166],
    [242, 167],
    [242, 168],
])

In [ ]:
az_sub = np.arange(227, 257)
rr_sub = np.arange(100, 175)

plt.figure(dpi=200)
# plt.pcolormesh(rr_sub, az_sub, z_marked[az_sub, :][:, rr_sub], cmap=cmap)
plt.pcolormesh(rr_sub, az_sub, z_rolled[az_sub, :][:, rr_sub], cmap=cmap)
plt.plot(cells2[:, 1] + 0.45, cells2[:, 0] + 0.45, 'xw', markersize=3)
plt.clim((-32, 96))
plt.grid()